In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
395  15691863  Female   46            41000          1
396  15706071    Male   51            23000          1
397  15654296  Female   50            20000          1
398  15755018    Male   36            33000          0
399  15594041  Female   49            36000          1

[400 rows x 5 columns]

In [4]:
# One-hot encoding categorical columns
dataset = pd.get_dummies(dataset, drop_first=True)


In [5]:
dataset

User ID  Age  EstimatedSalary  Purchased  Gender_Male
0    15624510   19            19000          0         True
1    15810944   35            20000          0         True
2    15668575   26            43000          0        False
3    15603246   27            57000          0        False
4    15804002   19            76000          0         True
..        ...  ...              ...        ...          ...
395  15691863   46            41000          1        False
396  15706071   51            23000          1         True
397  15654296   50            20000          1        False
398  15755018   36            33000          0         True
399  15594041   49            36000          1        False

[400 rows x 5 columns]

In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts() # Here we have to count the o's and 1's 
#Here we can check this BALANCED/IMBALANCED?
#If Balanced Dataset = It'll be in Equal numbers for both 0 & 1
# If Imbalanced = Deviations will be there.
# Unique Entries using .value_counts() method

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[['Age','EstimatedSalary','Purchased','Gender_Male']]
dep=dataset["Purchased"] # Here as usual we are not splitting the training set and test for Regression - GRID purpose.
#But here in the classification Grid we should spilt the training set and test set.
#If we not split the same Report will generate.  

In [9]:
indep.shape

(400, 4)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
# Split into training set and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
#Here We should add the Standard Scaler option as well

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() #if our predcitions will be under -2 to +2, Then we can keep it in the Standard Scaler
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
# htpps://scikit-learn.org/stable/modules/model_evaluation.html#storing-parameter

param_grid = {
    'criterion':['gini', 'entropy'],
    'max_features':['auto','sqrt','log2'],
    'splitter':['best','random']
}
# ROC => SVC(probability=True) We should give this probability then only We can do the 'ROC_AUC'
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3, cv=5, n_jobs = -1, scoring = 'f1_weighted') #It will run automatically and take the best model from the grid list
# We can do with RECALL & PRecision as well for the prediction(Which model is giving better performance)
# fitting the model for grid search
grid.fit(X_train, y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
17 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 436, in _validate_params
    validate_parameter_constraints(
    ~~~~~~~~~~~~

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [16]:
# Print the best parameter after tuning
# print (grid.best_params_)
re=grid.cv_results_
# print(re)
grid_predictions = grid.predict(X_test) # we can store the X_test in to gird.prediction

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_predictions, average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_), f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'best'}: 1.0


In [18]:
print("The Confusion Matrix:\n", cm)

The Confusion Matrix:
 [[85  0]
 [ 0 49]]


In [19]:
print("The Report:\n", clf_report)

The Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        85
           1       1.00      1.00      1.00        49

    accuracy                           1.00       134
   macro avg       1.00      1.00      1.00       134
weighted avg       1.00      1.00      1.00       134



In [20]:
from sklearn.metrics import roc_auc_score # [ROC = Receiver Operating Charactersitic Curve], [AUC = Area Under Curve]
roc_auc_score(y_test, grid.predict_proba(X_test)[:,1]) # proba => Parabola.
# where [:,1] => All rows in the first column
# : => represents the rows [We can take entire rows]
# And the 1 => Represents Column [first columns] 

np.float64(1.0)

In [21]:
table = pd.DataFrame.from_dict(re)

In [22]:
table

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.001090      0.000170         0.000000        0.000000   
1        0.000722      0.000044         0.000000        0.000000   
2        0.003625      0.002407         0.004460        0.001119   
3        0.001636      0.000058         0.003074        0.000293   
4        0.002264      0.000893         0.003790        0.001202   
5        0.001802      0.000460         0.003645        0.000850   
6        0.001090      0.000558         0.000000        0.000000   
7        0.001095      0.000530         0.000000        0.000000   
8        0.001918      0.000528         0.003199        0.000521   
9        0.002011      0.000703         0.003612        0.000976   
10       0.001719      0.000418         0.002832        0.000290   
11       0.001313      0.000846         0.002089        0.000727   

   param_criterion param_max_features param_splitter  \
0             gini               auto           best   
1             gini               auto         random   
2             gini               sqrt           best   
3             gini               sqrt         random   
4             gini               log2           best   
5             gini               log2         random   
6          entropy               auto           best   
7          entropy               auto         random   
8          entropy               sqrt           best   
9          entropy               sqrt         random   
10         entropy               log2           best   
11         entropy               log2         random   

                                               params  split0_test_score  \
0   {'criterion': 'gini', 'max_features': 'auto', ...                NaN   
1   {'criterion': 'gini', 'max_features': 'auto', ...                NaN   
2   {'criterion': 'gini', 'max_features': 'sqrt', ...           1.000000   
3   {'criterion': 'gini', 'max_features': 'sqrt', ...           1.000000   
4   {'criterion': 'gini', 'max_features': 'log2', ...           1.000000   
5   {'criterion': 'gini', 'max_features': 'log2', ...           1.000000   
6   {'criterion': 'entropy', 'max_features': 'auto...                NaN   
7   {'criterion': 'entropy', 'max_features': 'auto...                NaN   
8   {'criterion': 'entropy', 'max_features': 'sqrt...           1.000000   
9   {'criterion': 'entropy', 'max_features': 'sqrt...           1.000000   
10  {'criterion': 'entropy', 'max_features': 'log2...           1.000000   
11  {'criterion': 'entropy', 'max_features': 'log2...           0.981362   

    split1_test_score  split2_test_score  split3_test_score  \
0                 NaN                NaN                NaN   
1                 NaN                NaN                NaN   
2                 1.0                1.0           1.000000   
3                 1.0                1.0           1.000000   
4                 1.0                1.0           1.000000   
5                 1.0                1.0           0.981014   
6                 NaN                NaN                NaN   
7                 NaN                NaN                NaN   
8                 1.0                1.0           1.000000   
9                 1.0                1.0           1.000000   
10                1.0                1.0           1.000000   
11                1.0                1.0           0.981233   

    split4_test_score  mean_test_score  std_test_score  rank_test_score  
0                 NaN              NaN             NaN                9  
1                 NaN              NaN             NaN                9  
2            1.000000         1.000000        0.000000                1  
3            1.000000         1.000000        0.000000                1  
4            1.000000         1.000000        0.000000                1  
5            1.000000         0.996203        0.007594                6  
6                 NaN              NaN             NaN         

In [24]:
UserID_input=int(input("UserID:"))
Age_input=float(input("age:"))
EstimatedSalary_input=float(input("Estimated Salary:"))
Gender_Male_input=int(input("Gender Male 0 or 1:"))

UserID: 15624510
age: 19
Estimated Salary: 19000
Gender Male 0 or 1: 0


In [25]:
Future_Prediction=grid.predict([[UserID_input, Age_input, EstimatedSalary_input, Gender_Male_input]]) # Change the parameter according to your dataset
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


In [ ]:
e